<a href="https://colab.research.google.com/github/Abhishek0697/Detection-of-Hate-Speech-in-Multimodal-Memes/blob/main/Code/Experiments/BERT%2BResNeXt%2BImage_Captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd /content/gdrive/MyDrive/HatefulMemesDetection/Code/Experiments

Mounted at /content/gdrive/
/content/gdrive/MyDrive/HatefulMemesDetection/Code/Experiments


In [2]:

!nvidia-smi

Wed Apr 26 21:39:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!sudo apt-get update -y
!sudo apt-get install python3.7
!pip install torch transformers torchvision

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,334 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,035 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [2,341 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/ma

In [4]:
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
'''
IMPORTING NECESSARY MODULES
'''

import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset, TensorDataset
import sys
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

# sys.path.append('./Trainers/')
# sys.path.append('./Dataloaders/')
# sys.path.append('./utils/')
# sys.path.append('./architectures/')

sys.path.append('/content/gdrive/MyDrive/HatefulMemesDetection/Code/Trainers/')
sys.path.append('/content/gdrive/MyDrive/HatefulMemesDetection/Code/Dataloaders/')
sys.path.append('/content/gdrive/MyDrive/HatefulMemesDetection/Code/utils/')
sys.path.append('/content/gdrive/MyDrive/HatefulMemesDetection/Code/architectures/')
sys.path.append('/content/gdrive/MyDrive/HatefulMemesDetection/Code/Test/')

from dataloader import mydataset_captioning, mytestdataset
from Load_model import load
from plot_curves import plot_loss, plot_acc
from trainer_BERT_Captioning import train, test_classify

'''
For ResNeXt
'''
from resnet_models import ResNet,Bottleneck, resnext101_32x8d

'''
For BERT
'''
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

**Device**

In [6]:
# gpu_ids = [7,6]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


**Dataloading Scheme**

In [7]:
trainlist = '/content/gdrive/MyDrive/hm_project_pe/data/Generated_Caps/train_cap.jsonl'
validlist = '/content/gdrive/MyDrive/hm_project_pe/data/Generated_Caps/dev_cap.jsonl'
img_dir = '/content/gdrive/MyDrive/hm_project_pe/data'

In [8]:
'''
Train Dataloader
''' 
train_dataset = mydataset_captioning(annotations_file=trainlist, img_dir=img_dir, name='train')          
train_dataloader = data.DataLoader(train_dataset, shuffle= True, batch_size = 32, num_workers=16,pin_memory=True)


'''
Validation Dataloader
''' 
validation_dataset = mydataset_captioning(annotations_file=validlist, img_dir=img_dir, name='valid')         
validation_dataloader = data.DataLoader(validation_dataset, shuffle=False, batch_size = 32, num_workers=16,pin_memory=True)

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_ms

**Model Definition**

In [9]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [10]:
# '''
# Model1 ResNeXt101_32x8d
# '''
# # Image_model = ResNet(Bottleneck, [3, 4, 6, 3], num_classes = 2)
# Image_model = resnext101_32x8d()

# Image_model.fc = nn.Sequential(
#     nn.Linear(Image_model.fc.in_features, 2)
#     )

# Image_model = nn.DataParallel(Image_model,device_ids = [0]).to(device)


# '''
# Load saved model from checkpoint
# '''
# model1_name = 'ResneXt101_32x8d'
# model1_path = './saved_model_checkpoints/'+model1_name

# checkpoint1 = torch.load(model1_path)
# Image_model.load_state_dict(checkpoint1['model_state_dict'])
# Image_model.to(device)

In [11]:
'''
Model 2 BERT

Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
''' 

Text_model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 2,   
    output_attentions = False, 
    output_hidden_states = True
)

Text_model = nn.DataParallel(Text_model,device_ids=[0]).to(device)


'''
Load saved model from checkpoint
'''
model2_name = 'BERT_basic'
model2_path = './saved_model_checkpoints/'+model2_name

checkpoint2 = torch.load(model2_path)
Text_model.load_state_dict(checkpoint2['model_state_dict'])

Text_model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

DataParallel(
  (module): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, 

In [ ]:
'''
Model 3 BERT

Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top.
''' 

Text_model2 = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", 
    num_labels = 2,   
    output_attentions = False, 
    output_hidden_states = True
)

Text_model2 = nn.DataParallel(Text_model2,device_ids=[0]).to(device)


'''
Load saved model from checkpoint
'''
model2_name = 'BERT_Gen_Cap_basic'
model2_path = './saved_model_checkpoints/'+model2_name

checkpoint2 = torch.load(model2_path)
Text_model2.load_state_dict(checkpoint2['model_state_dict'])

Text_model2.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

DataParallel(
  (module): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, 

In [12]:
'''
Fusion 
Image Features, Text Features and Captions generated by our Captioning model
'''
class FusionNet(nn.Module):
    
    def __init__(self, num_classes, drop_prob = 0.1):
        super(FusionNet, self).__init__()
        
        self.concat = nn.Linear(in_features=768+768, out_features= 512)
        
        self.bn = nn.BatchNorm1d(512)
        self.bn1 = nn.BatchNorm1d(768)
        # self.bn2 = nn.BatchNorm1d(2048)
        self.bn3 = nn.BatchNorm1d(768)

        
        self.dropout = nn.Dropout(drop_prob)
        
        self.classify = nn.Linear(in_features = 512, out_features = num_classes)
        
        
    def forward(self, text_features, caption_features):

        text_features = self.bn1(text_features)
        # image_features = self.bn2(image_features)
        caption_features = self.bn3(caption_features)

        fused_input =  torch.cat((text_features, caption_features), dim=1)
        
        x = self.concat(fused_input)
        x = F.relu(self.bn(x))        
        
        x = F.relu(self.classify(x)) 

        return x


In [13]:
Fusion_model = FusionNet(num_classes = 2 , drop_prob = 0.1)
Fusion_model = nn.DataParallel(Fusion_model, device_ids=[0]).to(device)

**Hyperparameters**

In [14]:
'''
Loss Function
'''
criterion = nn.CrossEntropyLoss()


'''
Number of training epochs.
'''
num_Epochs = 10

lr_list = [0.1, 0.01, 0.001, 0.0001]
wd_list = [1e-4, 1e-5]

for lr in lr_list:
  for wd in wd_list:
    '''
    Optimizer
    '''
    print(lr, wd)
    optimizer = torch.optim.SGD(Fusion_model.parameters(), lr=lr, weight_decay=wd, momentum=0.9)
    # optimizer = AdamW(Fusion_model.parameters(), lr = 2e-3, eps = 1e-8)

    # '''
    # OneCycleLR
    # '''
    # max_lr = 0.05
    # lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, total_steps=None, epochs=num_Epochs, steps_per_epoch=len(train_dataloader), pct_start=0.3, anneal_strategy='cos', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=25.0, final_div_factor=10000.0, last_epoch=-1)


    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 4, gamma = 0.001)

    model_name = 'ImageCaptioning'
    model_path = './saved_model_checkpoints/'+model_name

    writer = SummaryWriter(model_name)

    train(Text_model, Text_model2, Fusion_model, train_dataloader, validation_dataloader, criterion, optimizer, lr_scheduler, model_path, writer, device, epochs = num_Epochs)

    writer.flush()
    writer.close()



    # lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=0, last_epoch=-1)

0.1 0.0001
Epoch:  1
training loss =  0.6870567740354323
Validation Loss: 0.6931	Top 1 Validation Accuracy: 0.5000
Epoch:  2
training loss =  0.6931474800396683
Validation Loss: 0.6931	Top 1 Validation Accuracy: 0.5000
Epoch:  3
training loss =  0.6931474800396683
Validation Loss: 0.6931	Top 1 Validation Accuracy: 0.5000
Epoch:  4
training loss =  0.6931474800396683
Validation Loss: 0.6931	Top 1 Validation Accuracy: 0.5000
Epoch:  5
training loss =  0.6931474800396683
Validation Loss: 0.6931	Top 1 Validation Accuracy: 0.5000
Epoch:  6
training loss =  0.6931474800396683
Validation Loss: 0.6931	Top 1 Validation Accuracy: 0.5000
Epoch:  7
training loss =  0.6931474800396683
Validation Loss: 0.6931	Top 1 Validation Accuracy: 0.5000
Epoch:  8
training loss =  0.6931474800396683
Validation Loss: 0.6931	Top 1 Validation Accuracy: 0.5000
Epoch:  9
training loss =  0.6931474800396683
Validation Loss: 0.6931	Top 1 Validation Accuracy: 0.5000
Epoch:  10
training loss =  0.6931474800396683
Valida

KeyboardInterrupt: ignored

In [ ]:
'''
Load saved model from checkpoint
'''
Fusion_model, optimizer, lr_scheduler, train_loss, v_loss, v_acc, epoch = load(model_path, Fusion_model, optimizer, lr_scheduler)


**Evaluate**

In [ ]:
test_loss, accuracy, original_label, predicted_label, prediction_probs = test_classify(Text_model, Text_model2, Fusion_model, validation_dataloader, criterion, device)

In [ ]:
import pandas as pd
original = pd.read_json("/content/gdrive/MyDrive/hm_project_pe/data/Generated_Caps/dev_cap.jsonl", lines=True)

In [ ]:
original.head()

,id,img,label,text,generated_caption
0,8291,img/08291.png,1,white people is this a shooting range,there are two young boys sitting on the ground...
1,46971,img/46971.png,1,bravery at its finest,there is a man eating a piece of food with a w...
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...,smiling woman behind a computer with a chalkbo...
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...,there is a picture of a knight holding a sword...
4,80243,img/80243.png,1,mississippi wind chime,a black and white photo of a man walking down ...


In [ ]:
original['original_label'] = original_label
original['predicted_label'] = predicted_label
original['prediction_probs'] = prediction_probs
original.to_csv('Bert_captioning_prediction.csv', index=False)

In [ ]:
original.head()

,id,img,label,text,generated_caption,original_label,predicted_label,prediction_probs
0,8291,img/08291.png,1,white people is this a shooting range,there are two young boys sitting on the ground...,1,0,0.952980
1,46971,img/46971.png,1,bravery at its finest,there is a man eating a piece of food with a w...,1,0,0.926696
2,3745,img/03745.png,1,your order comes to $37.50 and your white priv...,smiling woman behind a computer with a chalkbo...,1,0,0.500000
3,83745,img/83745.png,1,it is time.. to send these parasites back to t...,there is a picture of a knight holding a sword...,1,0,0.966576
4,80243,img/80243.png,1,mississippi wind chime,a black and white photo of a man walking down ...,1,0,0.500000


**Predict on Test and generate output.csv**

**Test Dataloader**

In [ ]:
testlist = 'path of test set'

test_dataset = mytestdataset(testlist, name='test')          
test_dataloader = data.DataLoader(test_dataset, shuffle= False, batch_size = 32, num_workers=8,pin_memory=True)

In [ ]:
from predict import predict
predict(image_model, text_model, fusion_model, test_dataloader, device)